In [269]:
import panel as pn
import pandas as pd
import numpy as np


file_input = pn.widgets.FileInput(accept='.csv', name='Upload CSV')

def load_csv(data):
    import io
    if data is not None:
        global df
        df = pd.read_csv(io.BytesIO(data))
        global orgin_lon, orgin_lat, dest_lon, dest_lat
        orgin_lon = pn.widgets.Select(name="Origin Lontitue", value="ZIP_lon", options=list(df.columns))
        orgin_lat = pn.widgets.Select(name="Origin Latitude", value="ZIP_lat", options=list(df.columns) )
        dest_lon = pn.widgets.Select(name="Destination Longitude", value="AHA_ID_lon", options=list(df.columns))
        dest_lat = pn.widgets.Select(name="Destination Latitude", value="AHA_ID_lat", options=list(df.columns))
        od_col_slect_view = pn.Column("## Select the columns for origin and destination", pn.Row(orgin_lon, orgin_lat, dest_lon, dest_lat))
        return pn.Column("## Preview of your data (first 5 rows, there are %d rows in total)"%df.shape[0],pn.pane.DataFrame(df.head(5)),
                         od_col_slect_view
                        )


active_load_csv = pn.bind(load_csv, file_input.param.value)


# pn.Column(file_input, active_load_csv)

data_upload_view = pn.Column("## Select data need to be calculated",file_input,active_load_csv)
data_upload_view

BokehModel(combine_events=True, render_bundle={'docs_json': {'49a490f1-d27a-43f5-ae68-8a95e15689df': {'version…

In [264]:
from georouting.routers import OSRMRouter
from panel.widgets import Tqdm
tqdm = Tqdm()
# create a router object 
router = OSRMRouter(mode="driving")

def calculate_distance(run):
    if not run:
        yield "Calculation did not run yet"
        return
    for k,v in tqdm(df.iterrows(), total=df.shape[0]):
        origin = (v[orgin_lon.value], v[orgin_lat.value])
        destination = (v[dest_lon.value], v[dest_lat.value])
        route = router.get_route(origin, destination)
        df.loc[k, 'distance (m)'] = route.get_distance()
        df.loc[k, 'duration (s)'] = route.get_duration()
    
    final_table = pn.pane.DataFrame(df.head(5), sizing_mode='stretch_width')
    from io import StringIO
    sio = StringIO()
    df.to_csv(sio)
    sio.seek(0)
    download_view = pn.widgets.FileDownload(sio, embed=True, filename='results.csv')
    table_download_view = pn.Column("## Processed data",download_view,final_table)
    
    yield table_download_view

# def runner(run):
#     if not run:
#         yield "Calculation did not run yet"
#         return
#     for i in range(101):
#         time.sleep(0.01) # Some calculation
#         yield pn.Column(
#             'Running ({i}/100%)', pn.indicators.Progress(value=i)
#         )
#     yield "Success ✅︎"


run = pn.widgets.Button(name="Press to run calculation")

# on_cli = button.on_click(calculate_distance)

run_and_download = pn.Column(run, tqdm, pn.bind(calculate_distance, run))




         
# pn.Column(button,tqdm)
# table_download_view = pn.bind(table_download, button.param.value)
# run_and_download = pn.Column(button,tqdm,table_download_view)
run_and_download

BokehModel(combine_events=True, render_bundle={'docs_json': {'a2d34ee1-3e43-4afa-a21c-15694c0457ca': {'version…

In [245]:
intro = pn.pane.Markdown("""
# Routing Calculator
This app is used to calculate the distance and duration between two points. This app is developed by [Xiaokang Fu](https://gis.harvard.edu/people/xiaokang-fu) and [Devika Kakkar](https://gis.harvard.edu/people/devika-kakkar).
Please contact [Devika Kakkar](mailto:kakkar@fas.harvard.edu) for any questions.
 """, 
sizing_mode='stretch_width'
)
intro

BokehModel(combine_events=True, render_bundle={'docs_json': {'adebe36b-7d82-42b9-ba19-1d45f1b4086d': {'version…

In [246]:

app = pn.Column(
       intro,
        data_upload_view,
        run_and_download
        )
app

BokehModel(combine_events=True, render_bundle={'docs_json': {'13add266-549c-4e44-aaa4-03c012fd9fce': {'version…

In [224]:
# app.show()

In [258]:
# Instantiate the template with widgets displayed in the sidebar
template = pn.template.FastListTemplate(
    title='Routing Calculator',
    logo='https://dssg.fas.harvard.edu/wp-content/uploads/2017/12/CGA_logo_globe_400x400.jpg',
    favicon = 'https://dssg.fas.harvard.edu/wp-content/uploads/2017/12/CGA_logo_globe_400x400.jpg',
    header_background = '#212121',
    header_color = '#2F6DAA',
)

# set relative sizing of the main area and sidebar

# Append a layout to the main area, to demonstrate the list-like API
template.main.append(app)
template.main.append(app)
template.servable()

Launching server at http://localhost:63560
